# Aufgabe 1 - Akku-Abenteuer: Tobi's Optimale Routenplanung

Den Code immer nachvollziehbar kommentieren! Bitte beachtet, dass das Notebook von Anfang bis Ende ohne Fehler durchlaufen muss und dass die requirements.txt Datei aktualisiert wird. 

## Teilaufgabe a): lageplan.png laden und verarbeiten

In [1]:
from matplotlib import image as mpimg
import numpy as np

# load picture
tempPlan = mpimg.imread("lageplanKleiner.png")
#Form: (21,21,4) - rows, columns, Array mit den vier Farbwerten (r,g,b,alpha)

tempPlan = np.around(tempPlan,2) #Runde alle Elemente der Matrix auf zwei Nachkommastellen

plan = [[0 for x in range(21)] for y in range(21)] #Erstelle 21 x 21 Matrix und initializiere mit 0

#Neuer Array plan enthält die selbe Info wie tempPlan, nur dass hier die Farben als String stehen. Außerdem ist die Matrix transponiert,
#damit die Koordinaten gleich den Koordinaten der Abbildung sind.
for m in range(21):
    for n in range(21):
        color = tempPlan[m][n]
        #print(color)
        if(color[0] == 0 and color[1] == 0 and color[2] == 0 and color[3] == 0 ):
            plan[n][m] = "white"
        elif(color[0] == 0 and color[1] == 0 and color[2] == 0 and color[3] == 1 ):
            plan[n][m] = "black"
        elif(abs(color[0] - 0.78) < 0.01 and abs(color[1] - 0.44) < 0.01 and abs(color[2] - 0.22) < 0.01 and color[3] == 1 ):
            plan[n][m] = "brown"
        elif(color[0] == 1 and color[1] == 1 and color[2] == 0 and color[3] == 1):
            plan[n][m] = "yellow"
        elif(color[0] == 0 and color[1] == 1 and color[2] == 0 and color[3] == 1):
            plan[n][m] = "green"
        elif(color[0] == 0 and color[1] == 0 and color[2] == 1 and color[3] == 1):
            plan[n][m] = "blue"
        #print(plan[n][m], end = " ")
    #print("\n")



## Teilaufgabe b): Breitensuche

In [21]:
#Gibt einen Array mit den Koordinaten der Nachbarn zurück. Nachbarn sind nicht diagonal. Nachbarn sind nicht schwarz oder weiß.
#Diese Methode wird von allen nachfolgenden Algorithmen benutzt
def getNeighbors(feld):
    neighbors = []

    #Füge alle Felder direkt neben dem Feld hinzu (nicht diagonal)
    if(feld[0] - 1 >= 0):
        neighbors.append([feld[0]-1,feld[1]])
    if(feld[0] + 1 < 21):
        neighbors.append([feld[0]+1,feld[1]])
    if(feld[1] - 1 >= 0):
        neighbors.append([feld[0],feld[1]-1])
    if(feld[1] + 1 < 21):
        neighbors.append([feld[0],feld[1]+1])

    #Lösche alle Nachbarn, die schwarz oder weiß sind
    for i in reversed(range(len(neighbors))):
        color = plan[neighbors[i][0]][ neighbors[i][1]]
        if(color == "black" or color == "white"):
            neighbors.pop(i)
    
    return neighbors

In [24]:
#start und ziel sind arrays mit Länge = 2
def breitensuche (start, ziel):
    # 21 x 21 Matrix. Enthält den jeweiligen Vorgängerknoten. Wird mit [-1,-1] initialisiert. Ein Eintrag != [-1,-1] bedeutet, dass der Knoten schon 
    # besucht wurde. Dann kann sofort zurückgesprungen werden.
    viewed = [[[-1,-1] for x in range(21)] for y in range(21)] 

    # Vorgänger des Startknotens wird auf einen einzigartigen erkennbaren Wert gesetzt: [25,25]
    viewed[start[0]][start[1]] = [25,25]
    
    queue = [start]

    while len(queue) > 0:
        node = queue.pop(0)
        #print("Aktuelle Node: ",node)
        
        if(node == ziel):
            print ("Weg zum Ziel: ",ziel, end = " <- ")
            prodecessor = viewed[node[0]][node[1]]
            while(prodecessor != [25,25]):
                print(prodecessor, end = " <- ")
                prodecessor = viewed[prodecessor[0]][prodecessor[1]]
                
            break

        neighbors = getNeighbors(node)
        for i in range(len(neighbors)):
            neighbor = neighbors[i]
            if(viewed[neighbor[0]][neighbor[1]] != [-1,-1]): #Breche ab, falls Nachbarfeld schon angeschaut wurde
                continue
            viewed[neighbor[0]][neighbor[1]] = node
            queue.append(neighbor)

breitensuche([3,17],[1,3])

Weg zum Ziel:  [1, 3] <- [1, 4] <- [1, 5] <- [1, 6] <- [1, 7] <- [2, 7] <- [3, 7] <- [3, 8] <- [3, 9] <- [3, 10] <- [3, 11] <- [3, 12] <- [3, 13] <- [2, 13] <- [1, 13] <- [1, 14] <- [1, 15] <- [1, 16] <- [1, 17] <- [2, 17] <- [3, 17] <- 

## Teilaufgabe c): A*-Algorithmus

In [19]:
#Genutzte Heuristik: Manhatten Metrik
    
#Knotenklasse. Wird in allen nachfolgenden Algorithmen genutzt
class Node:
    def __init__(self, coords, prodecessor, goal):
        self.coords = coords
        self.prodecessor = prodecessor

        #Manhatten-Metrik
        n1 = coords
        n2 = goal
        self.h = abs(n1[0]-n2[0]) + abs(n1[1]-n2[1])

        #Alles Folgende wird nur von A* genutzt:

        #Bestimme Kantenkosten (Kosten aller Kanten, die zu diesem Knoten zeigen)
        self.kantenkosten = 0
        #Die durch getNeighbors zurückgegebenen Knoten können weder weiß noch schwarz sein. Somit können wir die hier auslassen.
        color = plan[coords[0]][coords[1]]
        if(color == "brown"):
            kantenkosten = 2
        elif(color == "blue"): 
            kantenkosten = 3
        elif(color == "green"):
            kantenkosten = 4
        elif(color == "yellow"):
            kantenkosten = 5
            
        if(prodecessor == None):
            self.g = 0
        else:
            self.g = prodecessor.g + self.kantenkosten
            
        self.f = self.h + self.g   

In [33]:
#start und ziel sind arrays mit Länge = 2
def aStar (start, ziel):
    
    #Start Node ohne Vorgänger
    startNode = Node(start,None,ziel)
    
    #Enthält alle bereits expandierten Knoten
    closedList = []

    #Liste der besuchten, aber noch nicht expandierten Knoten
    openList = [startNode]
    
    while len(openList) > 0:
        #Bringe den Knoten mit der besten Abschätzung f = g + h nach vorne
        openList = sorted(openList, key=lambda x: x.f)
        
        #Nehme den Knoten mit der besten Abschätzung und entferne ihn aus der Liste
        node = openList.pop(0)
        
        #print("Aktuelle Node: ",node)
        
        if(node.coords == ziel):
            print ("Weg zum Ziel: ",ziel, end = " <- ")
            prodecessor = node.prodecessor
            while(prodecessor != None):
                print(prodecessor.coords, end = " <- ")
                prodecessor = prodecessor.prodecessor
                
            break

        #Füge Nachbarn in die openList ein, falls diese nicht in der closedList sind.
        neighbors = getNeighbors(node.coords)
        for i in range(len(neighbors)):
            neighborCoords = neighbors[i]
            if(neighborCoords in closedList): #Breche ab, falls Nachbarfeld schon angeschaut und expandiert wurde
                continue

            #Breche ab, falls Nachbarfeld schon angeschaut wurde oder vorherige Kantenkosten überschritten werden
            neighborNode = Node(neighborCoords, node, ziel)
            if(isInOpenListOrWorse(neighborNode, openList)): 
                continue
                
            openList.append(neighborNode)

        closedList.append(node.coords)

# Gibt True zurück, wenn Knoten nicht in der openList ist oder wenn vorherigen Kantenkosten unterschritten werden
def isInOpenListOrWorse(neighborNode, openList):
    for node in openList:
        if node.coords == neighborNode.coords:
            if node.g <= neighborNode.g:
                return True
            else:
                openList.remove(node)
    return False

aStar([3,17],[1,3])
    

Weg zum Ziel:  [1, 3] <- [1, 4] <- [1, 5] <- [1, 6] <- [1, 7] <- [2, 7] <- [3, 7] <- [3, 8] <- [3, 9] <- [3, 10] <- [3, 11] <- [3, 12] <- [3, 13] <- [2, 13] <- [1, 13] <- [1, 14] <- [1, 15] <- [1, 16] <- [1, 17] <- [2, 17] <- [3, 17] <- 

## Teilaufgabe d): Greedy Best First Search-Algorithmus

In [32]:
#start und ziel sind arrays mit Länge = 2
def greedy (start, ziel):
    
    #Start Node ohne Vorgänger
    startNode = Node(start,None,ziel)
    
    #Enthält alle bereits expandierten Knoten
    closedList = []
    
    #Liste der besuchten, aber noch nicht expandierten Knoten
    openList = [startNode]
    
    while len(openList) > 0:
        #Bringe den Knoten mit der besten Schätzung h nach vorne
        openList = sorted(openList, key=lambda x: x.h)
        
        #Nehme den Knoten mit der besten Schätzung und entferne ihn aus der Liste
        node = openList.pop(0)
        
        #print("Aktuelle Node: ",node)
        
        if(node.coords == ziel):
            print ("Weg zum Ziel: ",ziel, end = " <- ")
            prodecessor = node.prodecessor
            while(prodecessor != None):
                print(prodecessor.coords, end = " <- ")
                prodecessor = prodecessor.prodecessor
                
            break

        #Füge Nachbarn in die openList ein, falls diese nicht in der closedList sind.
        neighbors = getNeighbors(node.coords)
        for i in range(len(neighbors)):
            neighborCoords = neighbors[i]
            if(neighborCoords in closedList): #Breche ab, falls Nachbarfeld schon angeschaut und expandiert wurde
                continue
            if(isInOpenList(neighborCoords, openList)): #Breche ab, falls Nachbarfeld schon angeschaut wurde.
                continue
            neighborNode = Node(neighborCoords, node, ziel)
            openList.append(neighborNode)

        closedList.append(node.coords)

# nodeCoords ist ein Array
def isInOpenList(nodeCoords, openList):
    for node in openList:
        if node.coords == nodeCoords:
            return True
    return False
    

greedy([3,17],[1,3])
    

Weg zum Ziel:  [1, 3] <- [1, 4] <- [1, 5] <- [1, 6] <- [1, 7] <- [2, 7] <- [3, 7] <- [3, 8] <- [3, 9] <- [3, 10] <- [3, 11] <- [3, 12] <- [3, 13] <- [2, 13] <- [1, 13] <- [1, 14] <- [1, 15] <- [1, 16] <- [1, 17] <- [2, 17] <- [3, 17] <- 

## Teilaufgabe e): Dusseliger Doktorand

In [36]:
# Da nur die Kantengewichtung geändert wird, muss nur c) abgeändert werden. d) benutzt ausschließlich die Heuristik.
# Es muss nur die Berechnung der Kantenkosten in der Node Klasse geändert werden. 

class NodeE:
    def __init__(self, coords, prodecessor, goal):
        self.coords = coords
        self.prodecessor = prodecessor

        #Manhatten-Metrik
        n1 = coords
        n2 = goal
        self.h = abs(n1[0]-n2[0]) + abs(n1[1]-n2[1])

        #Alles Folgende wird nur von A* genutzt:

        if(prodecessor == None):
            self.g = 0
        else:
            #Bestimme Kantenkosten (Kosten aller Kanten, die zu diesem Knoten zeigen)
            self.kantenkosten = 0
            
            #Die durch getNeighbors zurückgegebenen Knoten können weder weiß noch schwarz sein. Somit können wir die hier auslassen.
            color = plan[coords[0]][coords[1]]
            if(color == "brown"):
                self.kantenkosten = 2
            elif(color == "blue"): 
                self.kantenkosten = 3
            elif(color == "green"):
                self.kantenkosten = 4
            elif(color == "yellow"):
                self.kantenkosten = 5
    
            if(coords[0] == 3 and prodecessor.coords[0] == 3):
                if(coords[1] < 15 and coords[1] > 2 and prodecessor.coords[1] < 15 and prodecessor.coords[1] > 2):
                    self.kantenkosten = 20
            
            self.g = prodecessor.g + self.kantenkosten
            
        self.f = self.h + self.g   


In [37]:
#A* Suche von oben kopiert und Node mit NodeE ausgetauscht


#start und ziel sind arrays mit Länge = 2
def aStarE (start, ziel):
    
    #Start Node ohne Vorgänger
    startNode = NodeE(start,None,ziel)
    
    #Enthält alle bereits expandierten Knoten
    closedList = []

    #Liste der besuchten, aber noch nicht expandierten Knoten
    openList = [startNode]
    
    while len(openList) > 0:
        #Bringe den Knoten mit der besten Abschätzung f = g + h nach vorne
        openList = sorted(openList, key=lambda x: x.f)
        
        #Nehme den Knoten mit der besten Abschätzung und entferne ihn aus der Liste
        node = openList.pop(0)
        
        #print("Aktuelle Node: ",node)
        
        if(node.coords == ziel):
            print ("Weg zum Ziel: ",ziel, end = " <- ")
            prodecessor = node.prodecessor
            while(prodecessor != None):
                print(prodecessor.coords, end = " <- ")
                prodecessor = prodecessor.prodecessor
                
            break

        #Füge Nachbarn in die openList ein, falls diese nicht in der closedList sind.
        neighbors = getNeighbors(node.coords)
        for i in range(len(neighbors)):
            neighborCoords = neighbors[i]
            if(neighborCoords in closedList): #Breche ab, falls Nachbarfeld schon angeschaut und expandiert wurde
                continue

            #Breche ab, falls Nachbarfeld schon angeschaut wurde oder vorherige Kantenkosten überschritten werden
            neighborNode = NodeE(neighborCoords, node, ziel)
            if(isInOpenListOrWorse(neighborNode, openList)): 
                continue
                
            openList.append(neighborNode)

        closedList.append(node.coords)

aStarE([3,17],[1,3])

Weg zum Ziel:  [1, 3] <- [2, 3] <- [3, 3] <- [3, 2] <- [3, 1] <- [4, 1] <- [5, 1] <- [6, 1] <- [7, 1] <- [8, 1] <- [9, 1] <- [10, 1] <- [11, 1] <- [12, 1] <- [13, 1] <- [14, 1] <- [15, 1] <- [16, 1] <- [17, 1] <- [18, 1] <- [19, 1] <- [19, 2] <- [19, 3] <- [19, 4] <- [19, 5] <- [19, 6] <- [19, 7] <- [19, 8] <- [19, 9] <- [19, 10] <- [19, 11] <- [19, 12] <- [19, 13] <- [19, 14] <- [18, 14] <- [17, 14] <- [16, 14] <- [15, 14] <- [14, 14] <- [13, 14] <- [12, 14] <- [11, 14] <- [10, 14] <- [9, 14] <- [8, 14] <- [7, 14] <- [6, 14] <- [5, 14] <- [5, 15] <- [5, 16] <- [4, 16] <- [3, 16] <- [3, 17] <- 